In [197]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from bayes_opt import BayesianOptimization
import category_encoders as ce
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor

In [198]:
train = pd.read_csv(r'train.csv', index_col=0)
test = pd.read_csv(r'test.csv', index_col=0)

In [199]:
#metrica
def MAE(actual, pred):
    return (np.mean(np.absolute(actual - pred)))

## Preprocesamiento

In [200]:
train.isnull().sum()

titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

In [201]:
test.isnull().sum()

titulo                         1378
descripcion                     401
tipodepropiedad                   7
direccion                     13191
ciudad                           83
provincia                        42
antiguedad                    10714
habitaciones                   5628
garages                        9323
banos                          6554
metroscubiertos                4299
metrostotales                 12655
idzona                         7179
lat                           30695
lng                           30695
fecha                             0
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
dtype: int64

In [202]:
train['provincia'] = train['provincia'].fillna('Distrito Federal') #valor mas frecuente
test['provincia'] = test['provincia'].fillna('Distrito Federal')
train['tipodepropiedad'] = train['tipodepropiedad'].fillna('Casa') #valor mas frecuente
test['tipodepropiedad'] = test['tipodepropiedad'].fillna('Casa')

train['descripcion'] = train['descripcion'].fillna('-')
test['descripcion'] = test['descripcion'].fillna('-')

train = train.fillna(0)
test = test.fillna(0)

In [203]:
print(train.shape)
print(test.shape)

(240000, 22)
(60000, 21)


## Feature Engineering

### One Hot Encoding

In [204]:
one_hot_enc = ce.OneHotEncoder()
one_hot_encoded = one_hot_enc.fit_transform(train['provincia'])
train = train.join(one_hot_encoded)
del train['provincia']

one_hot_encoded = one_hot_enc.transform(test['provincia'])
test = test.join(one_hot_encoded)
del test['provincia']

In [205]:
one_hot_enc = ce.OneHotEncoder()
one_hot_encoded = one_hot_enc.fit_transform(train['tipodepropiedad'])
train = train.join(one_hot_encoded)
del train['tipodepropiedad']

one_hot_encoded = one_hot_enc.transform(test['tipodepropiedad'])
test = test.join(one_hot_encoded)
del test['tipodepropiedad']

### Target Encoding

In [206]:
print(train.shape)
print(test.shape)

(240000, 76)
(60000, 75)


In [207]:
X = train.drop('precio', axis = 1)
y = train['precio']
target_enc = ce.TargetEncoder(cols=['idzona', 'ciudad'])
train = target_enc.fit_transform(X, y)
train['precio'] = y

In [208]:
test = target_enc.transform(test)

In [209]:
print(train.shape)
print(test.shape)

(240000, 76)
(60000, 75)


### Agregando Features

#### Avenida

In [210]:
def avenida(st):
    return ("av." in st) or ("avenida" in st) or  ("av " in st)

In [211]:
test = test.astype({"direccion":"str"})
train = train.astype({"direccion":"str"})

test["direccion"] = test["direccion"].apply(str.lower)
test["avenida"] = test["direccion"].map(avenida)

train["direccion"] = train["direccion"].apply(str.lower)
train["avenida"] = train["direccion"].map(avenida)

test = test.astype({"avenida":"uint8"})
train = train.astype({"avenida":"uint8"})

#### Cantidad de amenities

In [212]:
train['cant_amenities'] = train['usosmultiples'] + train['piscina'] + train['gimnasio']
test['cant_amenities'] = test['usosmultiples'] + test['piscina'] + test['gimnasio']

#### Año de publicación

In [213]:
train['fecha'] = pd.to_datetime(train['fecha'])
train['ano'] = train['fecha'].dt.year

test['fecha'] = pd.to_datetime(test['fecha'])
test['ano'] = test['fecha'].dt.year

#### De la descripción

In [214]:
buena_ubicacion = ['buena ubicacion', 'buena ubicación', 'excelente ubicación', 'excelente ubicacion', 'bien ubicada', 'bien ubicado']
train['buena_ubicacion'] = train['descripcion'].apply(lambda x: 1 if any(word in x for word in buena_ubicacion ) else 0)
test['buena_ubicacion'] = test['descripcion'].apply(lambda x: 1 if any(word in x for word in buena_ubicacion ) else 0)

In [215]:
luminoso = ['luminoso', 'luminosa']
train['luminoso'] = train['descripcion'].apply(lambda x: 1 if any(word in x for word in luminoso ) else 0)
test['luminoso'] = test['descripcion'].apply(lambda x: 1 if any(word in x for word in luminoso ) else 0)

In [216]:
jardin = ['jardin', 'jardín', 'parque', 'patio']
train['jardin'] = train['descripcion'].apply(lambda x: 1 if any(word in x for word in luminoso ) else 0)
test['jardin'] = test['descripcion'].apply(lambda x: 1 if any(word in x for word in luminoso ) else 0)

In [217]:
balcon = ['balcon', 'balcón', 'terraza']
train['balcon'] = train['descripcion'].apply(lambda x: 1 if any(word in x for word in luminoso ) else 0)
test['balcon'] = test['descripcion'].apply(lambda x: 1 if any(word in x for word in luminoso ) else 0)

#### Cercanías

In [218]:
train['centroscomercialescercanos'] = train['centroscomercialescercanos'].map({1:2, 0:0})
test['centroscomercialescercanos'] = test['centroscomercialescercanos'].map({1:2, 0:0})

In [219]:
train['cercanias'] = train['centroscomercialescercanos'] + train['escuelascercanas']
train['cercanias'] = train['cercanias'].astype(str)

test['cercanias'] = test['centroscomercialescercanos'] + test['escuelascercanas']
test['cercanias'] = test['cercanias'].astype(str)

In [220]:
train = train.join(pd.get_dummies(train['cercanias'], prefix = 'cercania'))
del train['cercanias']

test = test.join(pd.get_dummies(test['cercanias'], prefix = 'cercania'))
del test['cercanias']

### Borrando columnas innecesarias

In [221]:
del train['lat']
del train['lng']
del train['descripcion']
del train['titulo']
del train['direccion']
del train['fecha']

del test['lat']
del test['lng']
del test['descripcion']
del test['titulo']
del test['direccion']
del test['fecha']

In [222]:
print(train.shape)
print(test.shape)

(240000, 81)
(60000, 80)


### Dividiendo por año

In [223]:
train_2012 = train[train['ano'] == 2012]
train_2013 = train[train['ano'] == 2013]
train_2014 = train[train['ano'] == 2014]
train_2015 = train[train['ano'] == 2015]
train_2016 = train[train['ano'] == 2016]

test_2012 = test[test['ano'] == 2012]
test_2013 = test[test['ano'] == 2013]
test_2014 = test[test['ano'] == 2014]
test_2015 = test[test['ano'] == 2015]
test_2016 = test[test['ano'] == 2016]

## KNN

In [27]:
def KNN(train, param_grid):
    KNN = KNeighborsRegressor()
    X = train.drop(['precio'], axis=1)
    y = train['precio']
    grid = GridSearchCV(KNN, param_grid, cv=5, scoring='neg_mean_absolute_error')
    grid.fit(X, y)
    KNN = grid.best_estimator_
    print(grid.best_params_)
    print(grid.best_score_)
    return KNN

In [ ]:
k_valores = list(range(1, 21))
param_grid = dict(n_neighbors=k_valores)
KNN_2012 = KNN(train_2012, param_grid)
KNN_2013 = KNN(train_2013, param_grid)
KNN_2014 = KNN(train_2014, param_grid)
KNN_2015 = KNN(train_2015, param_grid)
KNN_2016 = KNN(train_2016, param_grid)

## Definiendo tamaños de los sets

In [224]:
def train_set(train,test):
    X = train.drop(['precio'], axis=1)
    y = train['precio']
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=1)
    print("Train shapes: X = " + str(X_train.shape) + " y = " + str(y_train.shape))
    print("Validation shapes: X = " + str(X_val.shape) +  " y = " + str(y_val.shape))
    print("Test shape: " + str(test.shape))
    return X, y, X_val, y_val

In [225]:
x_2012, y_2012, x_2012_val, y_2012_val = train_set(train_2012,test_2012)
x_2013, y_2013, x_2013_val, y_2013_val = train_set(train_2013,test_2013)
x_2014, y_2014, x_2014_val, y_2014_val = train_set(train_2014,test_2014)
x_2015, y_2015, x_2015_val, y_2015_val = train_set(train_2015,test_2015)
x_2016, y_2016, x_2016_val, y_2016_val = train_set(train_2016,test_2016)

Train shapes: X = (17650, 80) y = (17650,)
Validation shapes: X = (5884, 80) y = (5884,)
Test shape: (5899, 80)
Train shapes: X = (22789, 80) y = (22789,)
Validation shapes: X = (7597, 80) y = (7597,)
Test shape: (7592, 80)
Train shapes: X = (30429, 80) y = (30429,)
Validation shapes: X = (10143, 80) y = (10143,)
Test shape: (10018, 80)
Train shapes: X = (38602, 80) y = (38602,)
Validation shapes: X = (12868, 80) y = (12868,)
Test shape: (13017, 80)
Train shapes: X = (70528, 80) y = (70528,)
Validation shapes: X = (23510, 80) y = (23510,)
Test shape: (23474, 80)


## LIGHTGBM

### Tuneo de hiperparámetros

In [226]:
import lightgbm
def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, min_split_gain, min_child_weight, learning_rate, seed,n_estimators):
    params = {}
    params['metric'] = 'rmse'
    params["seed"] = int(seed)
    params["learning_rate"] = learning_rate
    params["num_leaves"] = int(round(num_leaves))
    params['feature_fraction'] = max(min(feature_fraction, 1), 0)
    params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
    params['max_depth'] = int(round(max_depth))
    params['min_split_gain'] = min_split_gain
    params['min_child_weight'] = min_child_weight
    params["n_estimators"] = int(n_estimators)
    cv_result = lightgbm.cv(params, dtrain, nfold=5, num_boost_round=100)
    # Used around 1000 boosting rounds in the full model
    #cv_result = lightgbm.cv(params, dtrain, num_boost_round=100, nfold=3, seed=20)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return max(cv_result['rmse-mean'])

In [227]:
def optimize_lgbm(sett):
    dtrain = sett
    lightgbm_bo = BayesianOptimization(lgb_eval, {'num_leaves': (24, 45),
                                        'feature_fraction': (0.1, 0.9),
                                        'bagging_fraction': (0.8, 1),
                                        'max_depth': (5, 8.99),
                                        'min_split_gain': (0.001, 0.1),
                                        'min_child_weight': (5, 50),
                                        "learning_rate" : (0.01, 0.07),
                                        "seed":(10,50),
                                        "n_estimators":(100,1000)          
                                        }, random_state=0)
    # Se probo con 3 puntos iniciales y 10 iteraciones, deberiamos probar con mas en ambas
    # La prox iteracion en casa probar 100 y 10000
    lightgbm_bo.maximize(init_points=3, n_iter=10, acq='ei')
    params = lightgbm_bo.max['params']
    params['max_depth'] = int(params['max_depth'])
    params["seed"] = int(params["seed"])
    params["n_estimators"] = int(params["n_estimators"])
    return params

In [231]:
feature_set = list(x_2012.columns)
dtrain_2012 = lightgbm.Dataset(x_2012, label=y_2012,feature_name=feature_set)
feature_set = list(x_2013.columns)
dtrain_2013 = lightgbm.Dataset(x_2013, label=y_2013,feature_name=feature_set)
feature_set = list(x_2014.columns)
dtrain_2014 = lightgbm.Dataset(x_2014, label=y_2014,feature_name=feature_set)
feature_set = list(x_2015.columns)
dtrain_2015 = lightgbm.Dataset(x_2015, label=y_2015,feature_name=feature_set)
feature_set = list(x_2016.columns)
dtrain_2016 = lightgbm.Dataset(x_2016, label=y_2016,feature_name=feature_set)


In [232]:
dtrain = dtrain_2012
params_2012 = optimize_lgbm(dtrain_2012)
dtrain = dtrain_2013
params_2013 = optimize_lgbm(dtrain_2013)
dtrain = dtrain_2014
params_2014 = optimize_lgbm(dtrain_2014)
dtrain = dtrain_2015
params_2015 = optimize_lgbm(dtrain_2015)
dtrain = dtrain_2016
params_2016 = optimize_lgbm(dtrain_2016)

|   iter    |  target   | baggin... | featur... | learni... | max_depth | min_ch... | min_sp... | n_esti... | num_le... |   seed    |
-------------------------------------------------------------------------------------------------------------------------------------


/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:503: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


|  1        |  1.727e+0 |  0.9098   |  0.6722   |  0.04617  |  7.174    |  24.06    |  0.06494  |  493.8    |  42.73    |  48.55    |


/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:503: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


|  2        |  1.739e+0 |  0.8767   |  0.7334   |  0.04173  |  7.266    |  46.65    |  0.008033 |  178.4    |  24.42    |  43.3     |


...

|  11       |  2.295e+0 |  0.951    |  0.6561   |  0.02132  |  8.973    |  41.21    |  0.04594  |  289.4    |  26.45    |  28.42    |


/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:503: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


|  12       |  2.303e+0 |  0.8457   |  0.4622   |  0.01581  |  6.111    |  28.36    |  0.05396  |  754.0    |  38.65    |  44.3     |


/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:503: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


|  13       |  2.253e+0 |  0.853    |  0.4545   |  0.0522   |  5.427    |  38.43    |  0.001316 |  103.9    |  33.05    |  13.05    |


In [234]:
print(params_2016)

{'bagging_fraction': 0.8844396091027046, 'feature_fraction': 0.2401644436304009, 'learning_rate': 0.019208260134863285, 'num_leaves': 30.282236771792782, 'n_estimators': 749, 'min_split_gain': 0.0010676751915985497, 'max_depth': 8, 'min_child_weight': 9.211388287076087, 'seed': 39}


In [240]:
from lightgbm import LGBMRegressor as lgbm
def fabrica_lgbm(params):
    best_lgbm_model = lgbm(#colsample_bytree=params["colsample_bytree"],
                 bagging_fracion=params["bagging_fraction"],
                 num_leaves=int(params["num_leaves"]),                 
                 learning_rate=params["learning_rate"],
                 max_depth=params["max_depth"],
                 min_child_weight=params["min_child_weight"],
                 n_estimators=params["n_estimators"],                                                                    
                 #reg_alpha=params["reg_alpha"],
                 #reg_lambda=params["reg_lambda"],
                 #subsample=params["subsample"],
                 seed=params["seed"])
    
    return best_lgbm_model

In [241]:
lgbm_2012 = fabrica_lgbm(params_2012)
lgbm_2013 = fabrica_lgbm(params_2013)
lgbm_2014 = fabrica_lgbm(params_2014)
lgbm_2015 = fabrica_lgbm(params_2015)
lgbm_2016 = fabrica_lgbm(params_2016)

In [243]:
lgbm_2012.fit(x_2012,y_2012)
lgbm_pred_val = lgbm_2012.predict(x_2012_val)
lgbm_mae = MAE(y_2012_val, lgbm_pred_val)
print("MAE XGB: " + str(lgbm_mae))

MAE XGB: 406024.91973549256


In [244]:
lgbm_2013.fit(x_2013,y_2013)
lgbm_pred_val = lgbm_2013.predict(x_2013_val)
lgbm_mae = MAE(y_2013_val, lgbm_pred_val)
print("MAE XGB: " + str(lgbm_mae))

MAE XGB: 407606.33905984863


In [245]:
lgbm_2014.fit(x_2014,y_2014)
lgbm_pred_val = lgbm_2014.predict(x_2014_val)
lgbm_mae = MAE(y_2014_val, lgbm_pred_val)
print("MAE XGB: " + str(lgbm_mae))

MAE XGB: 482616.50925858127


In [246]:
lgbm_2015.fit(x_2015,y_2015)
lgbm_pred_val = lgbm_2015.predict(x_2015_val)
lgbm_mae = MAE(y_2015_val, lgbm_pred_val)
print("MAE XGB: " + str(lgbm_mae))

MAE XGB: 534111.9699979337


In [247]:
lgbm_2016.fit(x_2016,y_2016)
lgbm_pred_val = lgbm_2016.predict(x_2016_val)
lgbm_mae = MAE(y_2016_val, lgbm_pred_val)
print("MAE XGB: " + str(lgbm_mae))

MAE XGB: 595856.9186336667


## XGBoost

### Tuneo de hiperparámetros

In [254]:
import xgboost as xgb

dtrain_2012 = xgb.DMatrix(x_2012, label=y_2012)
dtrain_2013 = xgb.DMatrix(x_2013, label=y_2013)
dtrain_2014 = xgb.DMatrix(x_2014, label=y_2014)
dtrain_2015 = xgb.DMatrix(x_2015, label=y_2015)
dtrain_2016 = xgb.DMatrix(x_2016, label=y_2016)

/usr/local/lib/python3.5/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.5/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [258]:
def xgb_evaluate(max_depth, gamma, colsample_bytree,seed,min_child_weight,n_estimators,
                  reg_alpha,reg_lambda,subsample,learning_rate):
    params = {'eval_metric': 'rmse',
              'max_depth': int(max_depth),
              'subsample': subsample,
              'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree,
              "seed": int(seed),
              "min_child_weight": min_child_weight,
              "n_estimators": n_estimators,
              "reg_alpha": reg_alpha,
              "reg_lambda": reg_lambda,
              "learning_rate": learning_rate
             }
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=3)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

In [259]:
def optimize(sett):
    dtrain = sett
    xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7), 
                                             'gamma': (0, 1),
                                             'colsample_bytree': (0.3, 0.9),
                                             "seed": (10,50),
                                             "min_child_weight": (0.4,1.5),
                                             "n_estimators":(100,10000),                                                                    
                                             "reg_alpha":(0.2,0.75),
                                             "reg_lambda": (0.2,0.8),
                                             "subsample" : (0.3, 0.8),
                                             "learning_rate": (0.01,0.07),
                                            })
    # Se probo con 3 puntos iniciales y 10 iteraciones, deberiamos probar con mas en ambas
    # La prox iteracion en casa probar 100 y 10000
    xgb_bo.maximize(init_points=3, n_iter=10, acq='ei')
    params = xgb_bo.max['params']
    params['max_depth'] = int(params['max_depth'])
    params["seed"] = int(params["seed"])
    params["n_estimators"] = int(params["n_estimators"])
    return params

In [260]:
dtrain = dtrain_2012
params_2012 = optimize(dtrain_2012)
dtrain = dtrain_2013
params_2013 = optimize(dtrain_2013)
dtrain = dtrain_2014
params_2014 = optimize(dtrain_2014)
dtrain = dtrain_2015
params_2015 = optimize(dtrain_2015)
dtrain = dtrain_2016
params_2016 = optimize(dtrain_2016)

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... |   seed    | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
|  1        | -8.862e+0 |  0.4441   |  0.6882   |  0.04941  |  5.864    |  1.301    |  8.159e+0 |  0.7123   |  0.3806   |  41.06    |  0.3587   |
|  2        | -8.73e+05 |  0.6479   |  0.3529   |  0.04546  |  5.359    |  1.227    |  7.129e+0 |  0.2772   |  0.4825   |  25.03    |  0.3723   |
|  3        | -8.795e+0 |  0.7166   |  0.1181   |  0.06087  |  4.716    |  1.335    |  2.768e+0 |  0.3284   |  0.6204   |  28.24    |  0.7622   |
|  4        | -9.164e+0 |  0.3032   |  0.363    |  0.04041  |  5.75     |  1.471    |  2.379e+0 |  0.3376   |  0.5848   |  13.38    |  0.4531   |
|  5        | -9.038e+0 |  0.5957   |  0.6237   |  0.03195  |  5.573    |  1.326    |  6.333e+0 |  0.4383   |  0.7137   |  4

|  4        | -9.728e+0 |  0.4439   |  0.8922   |  0.0427   |  5.509    |  1.03     |  965.0    |  0.749    |  0.7312   |  21.1     |  0.6454   |
|  5        | -9.232e+0 |  0.717    |  0.5619   |  0.04889  |  6.527    |  1.203    |  111.3    |  0.6511   |  0.3612   |  19.06    |  0.7976   |
|  6        | -9.649e+0 |  0.5067   |  0.2845   |  0.04345  |  5.758    |  1.144    |  5.929e+0 |  0.2273   |  0.2331   |  28.78    |  0.5094   |
|  7        | -1.019e+0 |  0.4393   |  0.4601   |  0.06556  |  3.353    |  1.277    |  2.859e+0 |  0.2258   |  0.3602   |  19.75    |  0.3014   |
|  8        | -9.258e+0 |  0.4626   |  0.713    |  0.05847  |  6.923    |  0.6994   |  9.935e+0 |  0.4273   |  0.2505   |  42.68    |  0.6172   |
|  9        | -1.143e+0 |  0.3266   |  0.8724   |  0.02944  |  3.267    |  1.319    |  6.808e+0 |  0.5571   |  0.3734   |  17.71    |  0.5421   |
|  10       | -1.185e+0 |  0.6589   |  0.5673   |  0.01693  |  5.432    |  0.415    |  8.168e+0 |  0.7106   |  0.5028   |  1

|  9        | -1.505e+0 |  0.3946   |  0.7737   |  0.01477  |  6.03     |  0.6256   |  7.582e+0 |  0.7055   |  0.5981   |  30.86    |  0.584    |
|  10       | -1.144e+0 |  0.3348   |  0.9027   |  0.03266  |  6.703    |  1.204    |  8.227e+0 |  0.2855   |  0.4947   |  37.05    |  0.4449   |
|  11       | -1.236e+0 |  0.6269   |  0.5188   |  0.03825  |  3.038    |  0.5808   |  7.788e+0 |  0.4467   |  0.2542   |  12.67    |  0.6322   |
|  12       | -1.154e+0 |  0.8981   |  0.297    |  0.06911  |  3.906    |  0.9029   |  4.14e+03 |  0.7258   |  0.5883   |  17.33    |  0.6235   |
|  13       | -1.119e+0 |  0.6203   |  0.1221   |  0.02842  |  6.071    |  1.052    |  1.751e+0 |  0.6864   |  0.635    |  43.22    |  0.4757   |


### Probando

In [261]:
def fabrica_XGB(params):
    best_xgb_model = XGBRegressor(colsample_bytree=params["colsample_bytree"],
                 gamma=params["gamma"],                 
                 learning_rate=params["learning_rate"],
                 max_depth=params["max_depth"],
                 min_child_weight=params["min_child_weight"],
                 n_estimators=params["n_estimators"],                                                                    
                 reg_alpha=params["reg_alpha"],
                 reg_lambda=params["reg_lambda"],
                 subsample=params["subsample"],
                 seed=params["seed"])
    
    return best_xgb_model

In [262]:
XGB_2012 = fabrica_XGB(params_2012)
XGB_2013 = fabrica_XGB(params_2013)
XGB_2014 = fabrica_XGB(params_2014)
XGB_2015 = fabrica_XGB(params_2015)
XGB_2016 = fabrica_XGB(params_2016)

In [263]:
XGB_2012.fit(x_2012,y_2012)
XGB_pred_val = XGB_2012.predict(x_2012_val)
XGB_mae = MAE(y_2012_val, XGB_pred_val)
print("MAE XGB: " + str(XGB_mae))

/usr/local/lib/python3.5/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.5/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[22:22:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE XGB: 176801.79285987423


In [264]:
XGB_2013.fit(x_2013,y_2013)
XGB_pred_val = XGB_2013.predict(x_2013_val)
XGB_mae = MAE(y_2013_val, XGB_pred_val)
print("MAE XGB: " + str(XGB_mae))

[22:26:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE XGB: 160516.67489017046


In [265]:
XGB_2014.fit(x_2014,y_2014)
XGB_pred_val = XGB_2014.predict(x_2014_val)
XGB_mae = MAE(y_2014_val, XGB_pred_val)
print("MAE XGB: " + str(XGB_mae))

[22:32:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE XGB: 508711.66124728875


In [266]:
XGB_2015.fit(x_2015,y_2015)
XGB_pred_val = XGB_2015.predict(x_2015_val)
XGB_mae = MAE(y_2015_val, XGB_pred_val)
print("MAE XGB: " + str(XGB_mae))

[22:33:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE XGB: 350874.9482388483


In [267]:
XGB_2016.fit(x_2016,y_2016)
XGB_pred_val = XGB_2016.predict(x_2016_val)
XGB_mae = MAE(y_2016_val, XGB_pred_val)
print("MAE XGB: " + str(XGB_mae))

[22:37:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE XGB: 318933.79442906607


In [272]:
print(params_2012)

{'subsample': 0.48520193050269284, 'learning_rate': 0.04954389117285075, 'colsample_bytree': 0.5542596325423537, 'reg_alpha': 0.21562617860171035, 'reg_lambda': 0.45926838232016043, 'n_estimators': 3462, 'seed': 28, 'max_depth': 6, 'min_child_weight': 0.9778725480963207, 'gamma': 0.7645394638651799}


In [273]:
print(params_2013)

{'subsample': 0.31535915678487986, 'learning_rate': 0.05113657454540983, 'colsample_bytree': 0.5494091757771876, 'reg_alpha': 0.34224452560602836, 'reg_lambda': 0.7186831927674757, 'n_estimators': 6031, 'seed': 14, 'max_depth': 6, 'min_child_weight': 0.5216582305373545, 'gamma': 0.5036817452851475}


## Random Forest

In [268]:
def fabrica_RF(train, param_grid):
    RF = RandomForestRegressor()
    X = train.drop(['precio'], axis=1)
    y = train['precio']
    grid = GridSearchCV(RF, param_grid, cv=4, scoring='neg_mean_absolute_error')
    grid.fit(X, y)
    RF = grid.best_estimator_
    print(grid.best_params_)
    print(grid.best_score_)
    return RF

In [276]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
    #'n_estimators': [200, 400]
    
}

In [277]:
rf_2012 = fabrica_RF(train_2012, param_grid)
rf_2013 = fabrica_RF(train_2013, param_grid)
rf_2014 = fabrica_RF(train_2014, param_grid)
rf_2015 = fabrica_RF(train_2015, param_grid)
rf_2016 = fabrica_RF(train_2016, param_grid)

{'bootstrap': True, 'n_estimators': 200, 'min_samples_leaf': 3, 'min_samples_split': 8, 'max_depth': 110, 'max_features': 3}
-685222.7626369509
{'bootstrap': True, 'n_estimators': 300, 'min_samples_leaf': 3, 'min_samples_split': 8, 'max_depth': 90, 'max_features': 3}
-659846.7173177076
{'bootstrap': True, 'n_estimators': 100, 'min_samples_leaf': 3, 'min_samples_split': 8, 'max_depth': 80, 'max_features': 3}
-742149.026873734
{'bootstrap': True, 'n_estimators': 300, 'min_samples_leaf': 3, 'min_samples_split': 8, 'max_depth': 80, 'max_features': 3}
-808693.6945384506
{'bootstrap': True, 'n_estimators': 200, 'min_samples_leaf': 3, 'min_samples_split': 8, 'max_depth': 100, 'max_features': 3}
-845238.0721589796


In [278]:
rf_2012 = RandomForestRegressor(n_estimators=200)
rf_2012.fit(x_2012,y_2012)
rf_pred_val = rf_2012.predict(x_2012_val)
rf_mae = MAE(y_2012_val, rf_pred_val)
print("MAE rf: " + str(rf_mae))

MAE rf: 169648.08362038207


In [280]:
rf_2013 = RandomForestRegressor(n_estimators=200)
rf_2013.fit(x_2013,y_2013)
rf_pred_val = rf_2013.predict(x_2013_val)
rf_mae = MAE(y_2013_val, rf_pred_val)
print("MAE rf: " + str(rf_mae))

MAE rf: 163430.72219079794


In [281]:
rf_2014 = RandomForestRegressor(n_estimators=200)
rf_2014.fit(x_2014,y_2014)
rf_pred_val = rf_2014.predict(x_2014_val)
rf_mae = MAE(y_2014_val, rf_pred_val)
print("MAE rf: " + str(rf_mae))

MAE rf: 187213.09548688054


In [282]:
rf_2015 = RandomForestRegressor(n_estimators=200)
rf_2015.fit(x_2015,y_2015)
rf_pred_val = rf_2015.predict(x_2015_val)
rf_mae = MAE(y_2015_val, rf_pred_val)
print("MAE rf: " + str(rf_mae))

MAE rf: 197270.41281697043


In [283]:
rf_2016 = RandomForestRegressor(n_estimators=200)
rf_2016.fit(x_2016,y_2016)
rf_pred_val = rf_2016.predict(x_2016_val)
rf_mae = MAE(y_2016_val, rf_pred_val)
print("MAE rf: " + str(rf_mae))

MAE rf: 211741.1321400569


In [105]:
#res_2012 = rf_2012.predict(test_2012)
#res_2013 = rf_2013.predict(test_2013)
#res_2014 = rf_2014.predict(test_2014)
#res_2015 = rf_2015.predict(test_2015)
#res_2016 = rf_2016.predict(test_2016)

## Ensembles

In [298]:
ensemble2012 = VotingRegressor([('rf', rf_2012), ('lgbm', lgbm_2012)])
ensemble2013 = VotingRegressor([('rf', rf_2013), ('lgbm', lgbm_2013)])
ensemble2014 = VotingRegressor([('rf', rf_2014), ('lgbm', lgbm_2014)])
ensemble2015 = VotingRegressor([('rf', rf_2015), ('lgbm', lgbm_2015)])
ensemble2016 = VotingRegressor([('rf', rf_2016), ('lgbm', lgbm_2016)])

In [299]:
ensemble2012.fit(x_2012, y_2012)
en_pred_val = ensemble2012.predict(x_2012_val)
en_mae = MAE(y_2012_val, en_pred_val)
print("MAE Ensemble: " + str(en_mae))

MAE Ensemble: 281303.0169945019


In [300]:
ensemble2013.fit(x_2013, y_2013)
en_pred_val = ensemble2013.predict(x_2013_val)
en_mae = MAE(y_2013_val, en_pred_val)
print("MAE Ensemble: " + str(en_mae))

MAE Ensemble: 279642.2977729409


In [301]:
ensemble2014.fit(x_2014, y_2014)
en_pred_val = ensemble2014.predict(x_2014_val)
en_mae = MAE(y_2014_val, en_pred_val)
print("MAE Ensemble: " + str(en_mae))

MAE Ensemble: 327702.9598104


In [302]:
ensemble2015.fit(x_2015, y_2015)
en_pred_val = ensemble2015.predict(x_2015_val)
en_mae = MAE(y_2015_val, en_pred_val)
print("MAE Ensemble: " + str(en_mae))

MAE Ensemble: 357964.5959041208


In [303]:
ensemble2016.fit(x_2016, y_2016)
en_pred_val = ensemble2016.predict(x_2016_val)
en_mae = MAE(y_2016_val, en_pred_val)
print("MAE Ensemble: " + str(en_mae))

MAE Ensemble: 394672.4225389817


In [304]:
res_2012 = ensemble2012.predict(test_2012)
res_2013 = ensemble2013.predict(test_2013)
res_2014 = ensemble2014.predict(test_2014)
res_2015 = ensemble2015.predict(test_2015)
res_2016 = ensemble2016.predict(test_2016)

In [305]:
res_2012 = pd.DataFrame(res_2012, index=test_2012.index, columns=['precio'])
res_2012 = res_2012.rename(columns={'precio':'target'})

res_2013 = pd.DataFrame(res_2013, index=test_2013.index, columns=['precio'])
res_2013 = res_2013.rename(columns={'precio':'target'})

res_2014 = pd.DataFrame(res_2014, index=test_2014.index, columns=['precio'])
res_2014 = res_2014.rename(columns={'precio':'target'})

res_2015 = pd.DataFrame(res_2015, index=test_2015.index, columns=['precio'])
res_2015 = res_2015.rename(columns={'precio':'target'})

res_2016 = pd.DataFrame(res_2016, index=test_2016.index, columns=['precio'])
res_2016 = res_2016.rename(columns={'precio':'target'})

In [306]:
res = res_2012.append(res_2013).append(res_2014).append(res_2015).append(res_2016)
res.to_csv("workshop-submission-rf.csv", header=True)